In [1]:
from tensorflow.python.client import device_lib   # 이는 나중에 GPU를 사용하고자 할때 필요함.
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 10847612422872064132
]


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns

In [27]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

In [4]:
import pandas as pd

df = pd.read_excel("Data for headed bars_for DataFrame_220725.xlsx", skiprows = 17, engine = 'openpyxl')
df = pd.DataFrame(df, columns = ["No.", "Author", "Year", "Test type", "Remark", "Specimen", "fy", "Ld", "fcm", "db", "b", "cos,avg",
                                 "cth", "ch", "Nh", "Bottom cover", "Ah/Ab", "Fsu at La, test", "dtr", "Ntr", "st"]) # st 제거시
df

,No.,Author,Year,Test type,Remark,Specimen,fy,Ld,fcm,db,...,"cos,avg",cth,ch,Nh,Bottom cover,Ah/Ab,"Fsu at La, test",dtr,Ntr,st
0,1.0,Wright,NaN,Stub-beam,NaN,1-TH01-P-NS-2DB-292,517.5,12.70,28.46,25.4,...,101.8,305.0,0.0,1.0,45.47,9.395954,376.153911,0.0,0.0,12.7
1,2.0,Wright,NaN,Stub-beam,NaN,1-TH02-P-NS-2DB-292,517.5,12.70,28.46,25.4,...,101.8,305.0,0.0,1.0,45.39,9.395954,388.271356,0.0,0.0,12.7
2,3.0,Wright,NaN,Stub-beam,NaN,1-TH03-P-NS-2DB-292,517.5,12.70,28.46,25.4,...,101.8,305.0,0.0,1.0,47.12,9.395954,364.470642,0.0,0.0,12.7
3,4.0,Wright,NaN,Stub-beam,NaN,2-TH04-NP-3S1-208-19,517.5,285.75,33.63,25.4,...,101.8,305.0,0.0,1.0,47.96,9.395954,482.092746,9.5,3.0,127.0
4,5.0,Wright,NaN,Stub-beam,NaN,2-TH05-NP-3S1-208-19,517.5,285.75,33.63,25.4,...,101.8,305.0,0.0,1.0,50.80,9.395954,462.692993,9.5,3.0,127.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
597,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
598,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
599,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
y= df["Fsu at La, test"]
y

0      376.153911
1      388.271356
2      364.470642
3      482.092746
4      462.692993
          ...    
597           NaN
598           NaN
599           NaN
600           NaN
601           NaN
Name: Fsu at La, test, Length: 602, dtype: float64

In [6]:
df.info()   # dtr, Ntr 데이터가 상대적으로 모자라니, 이 둘을 feature에서 제거함.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 602 entries, 0 to 601
Data columns (total 21 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   No.              431 non-null    float64
 1   Author           463 non-null    object 
 2   Year             398 non-null    float64
 3   Test type        463 non-null    object 
 4   Remark           206 non-null    object 
 5   Specimen         463 non-null    object 
 6   fy               463 non-null    float64
 7   Ld               463 non-null    float64
 8   fcm              463 non-null    float64
 9   db               463 non-null    float64
 10  b                463 non-null    float64
 11  cos,avg          463 non-null    float64
 12  cth              422 non-null    float64
 13  ch               431 non-null    float64
 14  Nh               463 non-null    float64
 15  Bottom cover     427 non-null    float64
 16  Ah/Ab            463 non-null    float64
 17  Fsu at La, test 

In [7]:
#X = df[["fy", "Ld", "fcm", "db", "b", "cos,avg", "cth", "ch", "Nh", "Bottom cover", "Ah/Ab", "dtr", "Ntr", "st"]] # 최대변수 사용
#X = df[["fy", "Ld", "fcm", "db", "b", "cos,avg", "cth", "ch", "Nh", "Bottom cover", "Ah/Ab", "st"]] # 누락값 많은 변수 미사용
# Test type - One-hot encoding
X = df[["Test type", "fy", "Ld", "fcm", "db", "b", "cos,avg", "cth", "ch", "Nh", "Bottom cover", "Ah/Ab", "st"]] # 범주형 데이터: Test type 포함
print(X["Test type"].value_counts()) # 범주 갯수 확인

Joint type    297
CCT            52
Lap splice     49
Stub-beam      33
Pullout        32
Name: Test type, dtype: int64


In [8]:
X = pd.get_dummies(data = X, columns = ["Test type"], prefix = "Test_type") # One-hot Endcoding 실행 -> (0, 0, 0, 1, 0), (1, 0, 0, 0, 0), ...
X

,fy,Ld,fcm,db,b,"cos,avg",cth,ch,Nh,Bottom cover,Ah/Ab,st,Test_type_CCT,Test_type_Joint type,Test_type_Lap splice,Test_type_Pullout,Test_type_Stub-beam
0,517.5,12.70,28.46,25.4,229.0,101.8,305.0,0.0,1.0,45.47,9.395954,12.7,0,0,0,0,1
1,517.5,12.70,28.46,25.4,229.0,101.8,305.0,0.0,1.0,45.39,9.395954,12.7,0,0,0,0,1
2,517.5,12.70,28.46,25.4,229.0,101.8,305.0,0.0,1.0,47.12,9.395954,12.7,0,0,0,0,1
3,517.5,285.75,33.63,25.4,229.0,101.8,305.0,0.0,1.0,47.96,9.395954,127.0,0,0,0,0,1
4,517.5,285.75,33.63,25.4,229.0,101.8,305.0,0.0,1.0,50.80,9.395954,127.0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
597,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0
598,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0
599,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0
600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0


In [9]:
y2 = y[~y.isnull()]
y2

0      376.153911
1      388.271356
2      364.470642
3      482.092746
4      462.692993
          ...    
473    457.241379
474    466.896552
475    412.413793
476    488.275862
477    459.310345
Name: Fsu at La, test, Length: 440, dtype: float64

In [10]:
X = X[~y.isnull()]
X

,fy,Ld,fcm,db,b,"cos,avg",cth,ch,Nh,Bottom cover,Ah/Ab,st,Test_type_CCT,Test_type_Joint type,Test_type_Lap splice,Test_type_Pullout,Test_type_Stub-beam
0,517.500000,12.70,28.460000,25.4,229.0,101.80,305.00,0.0,1.0,45.47,9.395954,12.7,0,0,0,0,1
1,517.500000,12.70,28.460000,25.4,229.0,101.80,305.00,0.0,1.0,45.39,9.395954,12.7,0,0,0,0,1
2,517.500000,12.70,28.460000,25.4,229.0,101.80,305.00,0.0,1.0,47.12,9.395954,12.7,0,0,0,0,1
3,517.500000,285.75,33.630000,25.4,229.0,101.80,305.00,0.0,1.0,47.96,9.395954,127.0,0,0,0,0,1
4,517.500000,285.75,33.630000,25.4,229.0,101.80,305.00,0.0,1.0,50.80,9.395954,127.0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
473,889.655172,154.94,35.724138,25.4,1524.0,769.62,769.62,NaN,1.0,NaN,5.100000,25.4,0,0,0,1,0
474,889.655172,172.72,37.655172,25.4,1524.0,769.62,769.62,NaN,1.0,NaN,5.100000,25.4,0,0,0,1,0
475,889.655172,160.02,37.655172,25.4,1524.0,774.70,774.70,NaN,1.0,NaN,5.100000,25.4,0,0,0,1,0
476,889.655172,167.64,37.655172,25.4,1524.0,762.00,762.00,NaN,1.0,NaN,5.100000,25.4,0,0,0,1,0


In [11]:
# X.fillna(0, inplace = True)   # 빈 데이터를을 모두 0으로 채우는 것은 잘못된 결과를 주어서 위험합니다. 차라리 다음과 같이 데이터를 없애는 게 낫습니다.
pd.options.display.max_rows=None

X.dropna(inplace = True)
X


C:\Users\BBamJoong\Anaconda3\envs\BBamJoong\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,fy,Ld,fcm,db,b,"cos,avg",cth,ch,Nh,Bottom cover,Ah/Ab,st,Test_type_CCT,Test_type_Joint type,Test_type_Lap splice,Test_type_Pullout,Test_type_Stub-beam
0,517.500000,12.700,28.460000,25.400,229.00,101.80,305.000000,0.000000,1.0,45.470000,9.395954,12.700,0,0,0,0,1
1,517.500000,12.700,28.460000,25.400,229.00,101.80,305.000000,0.000000,1.0,45.390000,9.395954,12.700,0,0,0,0,1
2,517.500000,12.700,28.460000,25.400,229.00,101.80,305.000000,0.000000,1.0,47.120000,9.395954,12.700,0,0,0,0,1
3,517.500000,285.750,33.630000,25.400,229.00,101.80,305.000000,0.000000,1.0,47.960000,9.395954,127.000,0,0,0,0,1
4,517.500000,285.750,33.630000,25.400,229.00,101.80,305.000000,0.000000,1.0,50.800000,9.395954,127.000,0,0,0,0,1
5,517.500000,292.100,33.630000,25.400,229.00,101.80,305.000000,0.000000,1.0,47.730000,9.395954,127.000,0,0,0,0,1
6,517.500000,289.052,33.630000,25.400,229.00,101.80,305.000000,0.000000,1.0,47.680000,9.395954,289.052,0,0,0,0,1
7,517.500000,289.052,33.630000,25.400,229.00,101.80,305.000000,0.000000,1.0,47.680000,9.395954,289.052,0,0,0,0,1
8,517.500000,289.052,33.630000,25.400,229.00,101.80,305.000000,0.000000,1.0,49.480000,9.395954,289.052,0,0,0,0,1
9,517.500000,285.750,33.440000,25.400,229.00,101.80,305.000000,0.000000,1.0,82.190000,9.395954,76.000,0,0,0,0,1


In [12]:
y2 = y2[X.index] # 목표값도 X와 동일하게 indexing
y2

0       376.153911
1       388.271356
2       364.470642
3       482.092746
4       462.692993
5       453.476630
6       293.542144
7       307.317351
8       330.585213
9       480.257367
10      486.138473
11      445.325971
12      451.562310
13      446.549556
14      447.417907
15      533.108373
16      518.622698
17      517.221495
18      524.859038
19      504.827757
20      485.526680
21      510.274686
22      499.834738
23      490.618375
24      492.631371
25      457.601298
26      507.472281
27      515.978174
28      520.813311
29      527.404886
30      490.697316
31      502.281909
32      523.280218
33      426.000000
34      456.000000
35      471.000000
36      451.000000
37      319.000000
38      386.000000
39      458.000000
40      458.000000
41      472.000000
42      472.000000
43        0.000000
44        0.000000
45      420.000000
46      472.000000
47      413.000000
48      458.000000
49      421.000000
50      459.000000
51      413.000000
52      471.

In [13]:
# y2 = 0인 값을 제거. 
y2 = y2.loc[(y2 != 0)]
y2

0       376.153911
1       388.271356
2       364.470642
3       482.092746
4       462.692993
5       453.476630
6       293.542144
7       307.317351
8       330.585213
9       480.257367
10      486.138473
11      445.325971
12      451.562310
13      446.549556
14      447.417907
15      533.108373
16      518.622698
17      517.221495
18      524.859038
19      504.827757
20      485.526680
21      510.274686
22      499.834738
23      490.618375
24      492.631371
25      457.601298
26      507.472281
27      515.978174
28      520.813311
29      527.404886
30      490.697316
31      502.281909
32      523.280218
33      426.000000
34      456.000000
35      471.000000
36      451.000000
37      319.000000
38      386.000000
39      458.000000
40      458.000000
41      472.000000
42      472.000000
45      420.000000
46      472.000000
47      413.000000
48      458.000000
49      421.000000
50      459.000000
51      413.000000
52      471.000000
53      376.000000
54      460.

In [14]:
X = X.loc[y2.index] # y=0인 index의 X값도 제거
X

,fy,Ld,fcm,db,b,"cos,avg",cth,ch,Nh,Bottom cover,Ah/Ab,st,Test_type_CCT,Test_type_Joint type,Test_type_Lap splice,Test_type_Pullout,Test_type_Stub-beam
0,517.500000,12.700,28.460000,25.400,229.00,101.80,305.000000,0.000000,1.0,45.470000,9.395954,12.700,0,0,0,0,1
1,517.500000,12.700,28.460000,25.400,229.00,101.80,305.000000,0.000000,1.0,45.390000,9.395954,12.700,0,0,0,0,1
2,517.500000,12.700,28.460000,25.400,229.00,101.80,305.000000,0.000000,1.0,47.120000,9.395954,12.700,0,0,0,0,1
3,517.500000,285.750,33.630000,25.400,229.00,101.80,305.000000,0.000000,1.0,47.960000,9.395954,127.000,0,0,0,0,1
4,517.500000,285.750,33.630000,25.400,229.00,101.80,305.000000,0.000000,1.0,50.800000,9.395954,127.000,0,0,0,0,1
5,517.500000,292.100,33.630000,25.400,229.00,101.80,305.000000,0.000000,1.0,47.730000,9.395954,127.000,0,0,0,0,1
6,517.500000,289.052,33.630000,25.400,229.00,101.80,305.000000,0.000000,1.0,47.680000,9.395954,289.052,0,0,0,0,1
7,517.500000,289.052,33.630000,25.400,229.00,101.80,305.000000,0.000000,1.0,47.680000,9.395954,289.052,0,0,0,0,1
8,517.500000,289.052,33.630000,25.400,229.00,101.80,305.000000,0.000000,1.0,49.480000,9.395954,289.052,0,0,0,0,1
9,517.500000,285.750,33.440000,25.400,229.00,101.80,305.000000,0.000000,1.0,82.190000,9.395954,76.000,0,0,0,0,1


In [15]:
##학습과 실험 데이터를 분류하고 train과 valid 데이터를 분류한뒤

In [16]:
# Train, Valid, Test Set으로 분류

In [17]:
# 1. Train + Valid : Test  = 0.9 : 0.1 --> 먼저 9:1로 나누고

In [18]:
from sklearn.model_selection import train_test_split

torch.manual_seed(1)

X_train, X_test, y_train, y_test = train_test_split(X, y2, test_size=0.20, random_state=142)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((288, 17), (73, 17), (288,), (73,))

In [19]:
# 2. Train : Valid = 8: 2  --> 9중 20%를 valid로 가져옴

In [20]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.20,random_state=142)

X_train.shape, X_valid.shape, y_train.shape, y_valid.shape

((230, 17), (58, 17), (230,), (58,))

In [21]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MaxAbsScaler


scX = MinMaxScaler()                    # X의 scaler 정의
X_train_scaled = scX.fit_transform(X_train)    
X_valid_scaled = scX.transform(X_valid)
X_test_scaled = scX.transform(X_test)

scY = MinMaxScaler()                    # y의 scaler 정의
y_train_scaled = scY.fit_transform(y_train.values.reshape(-1,1)) # y_train.values.reshape(-1,1) 는 Pandas Series를 조작하여 차원조절
y_valid_scaled = scY.transform(y_valid.values.reshape(-1,1))
y_test_scaled = scY.transform(y_test.values.reshape(-1,1))

In [22]:
# 텐서 데이터로 변환하는 클래스
class TensorData(Dataset):

    def __init__(self, x_data, y_data):
        self.x_data = torch.FloatTensor(x_data)
        self.y_data = torch.FloatTensor(y_data)
        self.len = self.y_data.shape[0]

    def __getitem__(self, index):

        return self.x_data[index], self.y_data[index] 

    def __len__(self):
        return self.len

In [23]:
trainsets = TensorData(X_train_scaled, y_train_scaled)
trainloader = torch.utils.data.DataLoader(trainsets, batch_size=32, shuffle=True)

testsets = TensorData(X_test_scaled, y_test_scaled)
testloader = torch.utils.data.DataLoader(testsets, batch_size=32, shuffle=False)

In [24]:
x, y = next(iter(trainloader))

X_train_scaled.shape
y_train_scaled.shape

(230, 1)

In [25]:
class Regressor(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(17, 3000, bias=True)
        self.fc2 = nn.Linear(3000, 1750, bias=True)
        self.fc3 = nn.Linear(1750, 625, bias=True)
        self.fc4 = nn.Linear(625, 100, bias=True)
        self.fc5 = nn.Linear(100, 1, bias=True)

    def forward(self, x): # 모델 연산의 순서를 정의
        x = F.relu(self.fc1(x)) 
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = self.fc5(x)
      
        return x     
    

In [28]:
model = Regressor()
criterion = nn.MSELoss()

optimizer = optim.Adam(model.parameters(), lr = 0.001, weight_decay = 1e-7)

In [33]:
loss_ = [] # 그래프를 그리기 위한 loss 저장용 리스트 
n = len(trainloader)

for epoch in range(1000): # 1000번 학습을 진행한다.

    running_loss = 0.0
    
    for data in trainloader: # 무작위로 섞인 32개 데이터가 있는 배치가 하나 씩 들어온다.

        inputs, values = data # input은 X값, values는 Y값

        optimizer.zero_grad() # 최적화 초기화
        
        outputs = model(inputs) # 모델에 입력값 대입 후 예측값 산출
        loss = criterion(outputs, values) # 손실 함수 계산
        loss.backward() # 손실 함수 기준으로 역전파 설정 
        optimizer.step() # 역전파를 진행하고 가중치 업데이트
        
        running_loss += loss.item() # epoch 마다 평균 loss를 계산하기 위해 배치 loss를 더한다.

    loss_.append(running_loss/n) # MSE(Mean Squared Error) 계산

        
print('Finished Training')

tensor(0.0005, grad_fn=<MseLossBackward0>)
tensor(0.0006, grad_fn=<MseLossBackward0>)
tensor(0.0004, grad_fn=<MseLossBackward0>)
tensor(0.0003, grad_fn=<MseLossBackward0>)
tensor(0.0003, grad_fn=<MseLossBackward0>)
tensor(0.0005, grad_fn=<MseLossBackward0>)
tensor(0.0004, grad_fn=<MseLossBackward0>)
tensor(0.0008, grad_fn=<MseLossBackward0>)
tensor(0.0004, grad_fn=<MseLossBackward0>)
tensor(0.0006, grad_fn=<MseLossBackward0>)
tensor(0.0004, grad_fn=<MseLossBackward0>)
tensor(0.0006, grad_fn=<MseLossBackward0>)
tensor(0.0004, grad_fn=<MseLossBackward0>)
tensor(0.0008, grad_fn=<MseLossBackward0>)
tensor(0.0006, grad_fn=<MseLossBackward0>)
tensor(0.0002, grad_fn=<MseLossBackward0>)
tensor(0.0004, grad_fn=<MseLossBackward0>)
tensor(0.0003, grad_fn=<MseLossBackward0>)
tensor(0.0004, grad_fn=<MseLossBackward0>)
tensor(0.0008, grad_fn=<MseLossBackward0>)
tensor(0.0003, grad_fn=<MseLossBackward0>)
tensor(0.0006, grad_fn=<MseLossBackward0>)
tensor(0.0006, grad_fn=<MseLossBackward0>)
tensor(7.06

KeyboardInterrupt: 

In [32]:
loss_

[0.12401046860031784,
 0.041439777007326484,
 0.021617791207972914,
 0.01922714977990836,
 0.017826947034336627,
 0.016411887772846967,
 0.01393135788384825,
 0.012541148229502141,
 0.013158519985154271,
 0.010774566093459725,
 0.01763486850541085,
 0.01126027840655297,
 0.01022073085187003,
 0.011810846801381558,
 0.0075108146702405065,
 0.014398683910258114,
 0.011274046963080764,
 0.013528400973882526,
 0.009937179100234061,
 0.006936783145647496,
 0.006526465178467333,
 0.007089639664627612,
 0.00506937361205928,
 0.00634931109379977,
 0.005588719359366223,
 0.004393409384647384,
 0.0047132491308730096,
 0.0048235950380330905,
 0.004875177954090759,
 0.008691619761520997,
 0.006721651792759076,
 0.006167020939756185,
 0.004219167603878304,
 0.005882539291633293,
 0.00814928745967336,
 0.0044513483153423294,
 0.003912086482159793,
 0.0033669919066596776,
 0.0044526549463626,
 0.004657619982026517,
 0.005185465968679637,
 0.004433507230714895,
 0.00488737435080111,
 0.004219009657390

In [31]:
a = 0
for i in range(len(loss_)):
    a += loss_[i]
    
a/len(loss_)

0.001666352062381293

In [ ]:
plt.plot(loss_)
plt.title("Training Loss")
plt.xlabel("epoch")
plt.show()

In [ ]:
def evaluation(dataloader):
    
    predictions = torch.tensor([], dtype=torch.float) # 예측값을 저장하는 텐서
    actual = torch.tensor([], dtype=torch.float) # 실제값을 저장하는 텐서
        
    with torch.no_grad():
        model.eval() # 평가를 할 때에는 .eval() 반드시 사용해야 한다.
        for data in dataloader:
            inputs, values = data
            outputs = model(inputs)

            predictions = torch.cat((predictions, outputs), 0) # cat을 통해 예측값을 누적
            actual = torch.cat((actual, values), 0) # cat을 통해 실제값을 누적

    rmse = np.sqrt(mean_squared_error(predictions, actual)) # sklearn을 이용하여 RMSE 계산
    
    return rmse

In [ ]:
from sklearn.metrics import mean_squared_error

train_rmse = evaluation(trainloader) # 학습 데이터의 RMSE
test_rmse = evaluation(testloader) # 시험 데이터의 RMSE

print("Train RMSE: ",train_rmse)
print("Test RMSE: ",test_rmse)

In [ ]:
def prediction(dataloader):
    
    predictions = torch.tensor([], dtype=torch.float) # 예측값을 저장하는 텐서
        
    with torch.no_grad():
        model.eval() # 평가를 할 때에는 .eval() 반드시 사용해야 한다.
        for data in dataloader:
            inputs, values = data
            outputs = model(inputs)

            predictions = torch.cat((predictions, outputs), 0) # cat을 통해 예측값을 누적
    
    return predictions

In [ ]:
y_p = prediction(testloader).numpy()


y_test_unscaled = scY.inverse_transform(y_test_scaled)
y_p_unscaled = scY.inverse_transform(y_p)



from sklearn.metrics import r2_score
r2 = r2_score(y_test_unscaled, y_p_unscaled)

division = y_p_unscaled / y_test_unscaled
cov = np.std(division) / np.mean(division)

In [ ]:
print("R2_score : %f" %r2)
print("cov : %f" %cov)